In [26]:
import arcpy
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()
gdb_path = os.getenv('GDB_PATH')
# Convert the file path to a raw string
gdb_path = r"{}".format(gdb_path)
feature_class_path = os.path.join(gdb_path,"feature_class1")
feature_class_path = r"{}".format(feature_class_path)

In [ ]:
### Check that both the geodatabase and feature class exist
if arcpy.Exists(gdb_path): 
    print(f"Geodatabase exists: {gdb_path}")
if arcpy.Exists(feature_class_path):
    print(f"Feature Class exists: {feature_class_path}" )

In [30]:

### Get all the paths to the json coordinate data
grocery_data_path = Path("..\data_collection\grocery.json")
convenience_data_path = Path("..\data_collection\convenience.json")
cornerstore_data_path = Path("..\data_collection\cornerstore.json")
drugs_nliquor_data_path = Path("..\data_collection\drugs_nliquor.json")
farm_market_data_path = Path("..\data_collection\\farm_market.json")
fast_food_data_path = Path("..\data_collection\\fast_food.json")
food_bank_data_path = Path("..\data_collection\\food_bank.json")
mini_mart_data_path = Path("..\data_collection\mini_mart.json")

DATA_PATH_LIST = [grocery_data_path,convenience_data_path,
                  cornerstore_data_path,
                  drugs_nliquor_data_path,
                  farm_market_data_path,
                  fast_food_data_path,
                  food_bank_data_path,
                  mini_mart_data_path
                  ]

for path in DATA_PATH_LIST:
    if path.exists():
        print(f"{path.name} exists")
    else:
        print(f"{path.name} doesn't exist")


grocery.json exists
convenience.json exists
cornerstore.json exists
drugs_nliquor.json exists
farm_market.json exists
fast_food.json exists
food_bank.json exists
mini_mart.json exists


In [ ]:
### Load in json data and plot on feature layer
import json
import arcpy
from arcpy.da import InsertCursor
from shapely.geometry import Point

spatial_ref = arcpy.SpatialReference(4326) # WGS 84 (latitude/longitude)
FEATURE_CLASS = "feature_class1"

In [ ]:
grocery_json = json.loads(grocery_data_path)
grocery_fields = ['@SHAPE','Address']

# Create an insert cursor to add data to the feature class
with InsertCursor(FEATURE_CLASS,grocery_fields) as cursor:
    for address, coordinates in grocery_json.items():
        lat = coordinates[0]
        lon = coordinates[1]

        # Create a point geometry (WGS84)
        point = arcpy.Point(lon, lat)
        point_geometry = arcpy.PointGeometry(point, spatial_ref)

        # Insert the point into the feature class
        cursor.insertRow([point_geometry, address])